In [2]:
#Using most of the code from kaggle kernel platform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA, FastICA
%matplotlib inline
n_comp = 10
color = sns.color_palette()

In [3]:
train_df = pd.read_csv('train.csv')
test_df =  pd.read_csv('test.csv')
print("Train shape:",train_df.shape)
print("Test shape:",test_df.shape);

Train shape: (4209, 378)
Test shape: (4209, 377)


In [4]:
for c in train_df.columns:
    if train_df[c].dtype == "object":
        lbl = LabelEncoder()
        lbl.fit(list(train_df[c].values) + list(test_df[c].values))
        train_df[c] = lbl.transform(list(train_df[c].values))
        test_df[c]  = lbl.transform(list(test_df[c].values))

pca = PCA(n_components=n_comp, random_state=93)
pca_train = pca.fit_transform(train_df.drop(["y","ID"],axis=1))
pca_test  = pca.transform(test_df.drop("ID",axis=1))
pca_train = pd.DataFrame(columns=["pca_%d" % (i) for i in range(n_comp)],data=pca_train)
pca_test = pd.DataFrame(columns=["pca_%d" % (i) for i in range(n_comp)],data=pca_test)

ica = FastICA(n_components=n_comp, random_state=93)
ica_train = ica.fit_transform(train_df.drop(["y","ID"],axis=1))
ica_test  = ica.transform(test_df.drop("ID",axis=1))
ica_train = pd.DataFrame(columns=["ica_%d" % (i) for i in range(n_comp)],data=ica_train)
ica_test = pd.DataFrame(columns=["ica_%d" % (i) for i in range(n_comp)],data=ica_test)

train_feat = pd.concat([pca_train,ica_train],axis=1)
test_feat = pd.concat([pca_test,ica_test],axis=1)

from sklearn.preprocessing import MinMaxScaler

mmfeat = MinMaxScaler(feature_range=(-1,1))
feat = mmfeat.fit_transform(pd.concat([train_feat,test_feat]))

train_feat = pd.DataFrame(columns=train_feat.columns, data=feat[:train_feat.shape[0],:])
test_feat = pd.DataFrame(columns=test_feat.columns, data=feat[test_feat.shape[0]:,:])

y = train_df['y'].values
mmy = MinMaxScaler(feature_range=(-1,1))
y = mmy.fit_transform(y.reshape(-1,1))

In [5]:
from sklearn import svm
clf = svm.SVR()
clf.fit(train_feat.values, np.ravel(y))
pred = clf.predict(test_feat.values)
pred = mmy.inverse_transform(pred.reshape(-1,1))

res = pd.DataFrame(columns = ['ID','y'], data=np.concatenate((test_df['ID'].values.reshape(-1,1),pred),axis=1))
res['ID'] = res['ID'].astype('int')
print(res.head())
res.to_csv(path_or_buf='submission.csv',index=False)

   ID           y
0   1   83.094222
1   2  100.313241
2   3   91.769344
3   4   84.882721
4   5  106.350002
